# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [2]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [3]:
import json
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # sklearn 내 confusion matrix 계산 함수
import matplotlib.pyplot as plt # 시각화를 위한 라이브러리
import torchvision.transforms as T # 이미지 변환을 위한 모듈

In [4]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [5]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [6]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [7]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '/data/ephemeral/home/datasets_fin/'

# model config
model_name = 'efficientnet_b3.ra2_in1k'#'densenet121.ra_in1k'# #'resnet101' #'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 256
LR = 1e-3
EPOCHS = 1
BATCH_SIZE = 32
num_workers = 0

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [49]:
# https://lcyking.tistory.com/entry/Albumentations%EC%9D%98-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%A6%9D%EA%B0%95-%EC%9D%B4%ED%95%B4
from PIL import Image
import numpy as np
import torch
from albumentations import Compose, Resize, RandomCrop, HorizontalFlip, Rotate, VerticalFlip, RandomRotate90, ShiftScaleRotate, GaussNoise, Cutout, CLAHE, Blur

def create_augmentations():
    return Compose([
        #Resize(height=224, width=224),
        #RandomCrop(height=200, width=200), 사용불가
        #Rotate(limit=90, p=1), #1
        #HorizontalFlip(p=0.5), #2
        #VerticalFlip(p=0.5), #3
        #RandomRotate90(p=1), #4
        #ShiftScaleRotate(shift_limit=0, scale_limit=(0.9, 1), rotate_limit=20, p=1), 사용불가
        #ShiftScaleRotate(shift_limit=0, scale_limit=(0.5, 0.9), rotate_limit=0, p=1), # Only_Scale
        #ShiftScaleRotate(shift_limit=(0.4, 0.5), scale_limit=0, rotate_limit=0, p=1), # Only_Shift
        GaussNoise(p=1, var_limit=(600, 800)), #5
        #Cutout(p=1, num_holes=8, max_h_size=24, max_w_size=24),
        #Blur(p=1, blur_limit=(3, 5)) #6
        #CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1) 사용안함
    ])

def to_tensor(image_np):
    return torch.tensor(image_np).permute(2, 0, 1).float() / 255.0

def augment_and_save_image(image_path, save_dir, num_copies=1):
    image = Image.open(image_path).convert('RGB')  # RGBA를 RGB로 명시적 변환
    image_np = np.array(image)

    if image_np.dtype != np.uint8:
        print(f"Data type mismatch in image {image_path}, found {image_np.dtype}, expected uint8.")
        return
    
    augmentation = create_augmentations()
    augmented_images = [augmentation(image=image_np) for _ in range(num_copies)]

    for i, aug in enumerate(augmented_images):
        tensor_image = to_tensor(aug['image'])
        image_np = tensor_image.permute(1, 2, 0).numpy()  # CHW -> HWC
        image_np = (image_np * 255).astype(np.uint8)  # 데이터 스케일 복구
        augmented_image = Image.fromarray(image_np)  # numpy 배열을 PIL 이미지로 변환
        save_path = os.path.join(save_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_9.jpg")
        augmented_image.save(save_path)  # 이미지 저장

# 사용 예시
import os

image_dir = '/data/ephemeral/home/datasets_fin/HorizontalFlip'
save_dir = '/data/ephemeral/home/datasets_fin/HorizontalFlip_GaussNoise'
if not os.path.exists(save_dir):
     os.makedirs(save_dir)

for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        augment_and_save_image(os.path.join(image_dir, filename), save_dir)
